In [1]:
import pandas as pd

df = pd.read_csv('./ml-1m/ratings.dat', sep='::', names=['userId', 'movieId', 'rating', 'timestamp'])
df

<ipython-input-1-f7cc5735a2f1>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('./ml-1m/ratings.dat', sep='::', names=['userId', 'movieId', 'rating', 'timestamp'])


,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [2]:
d = pd.DataFrame(df.userId.value_counts())
d.columns = ['nums']

d = d[d['nums'] < 50]
delindexs = d.index
print(len(delindexs))

df = df[~df['userId'].isin(delindexs)]

# df['movieId'].value_counts()
df

1743


,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [3]:
df['movieId'].value_counts()

d = pd.DataFrame(df.movieId.value_counts())
d.columns = ['nums']

d = d[d['nums'] < 50]
delindexs = d.index
print(len(delindexs))

df = df[~df['movieId'].isin(delindexs)]
df


1212


,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [2]:
df.to_csv('./ml-1m/rating_p.csv',index=0)

In [3]:
df = pd.read_csv('./ml-1m/rating_p.csv')
df

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [4]:

user_df = df.drop_duplicates(['userId'])
user = list(user_df['userId'])
id = list(range(len(user_df)))

user_list = {
    "org_id" : user,
    "remap_id" : id
}
user_list = pd.DataFrame(user_list)

user_list.to_csv('./user_list.txt', sep=' ', index=False)

user_list

,org_id,remap_id
0,1,0
1,2,1
2,3,2
3,4,3
4,5,4
...,...,...
6035,6036,6035
6036,6037,6036
6037,6038,6037
6038,6039,6038


In [5]:
item_df = df.drop_duplicates(['movieId']) #删除数据记录中movieId列值相同的记录
item_df = item_df.sort_values('movieId')
item = list(item_df['movieId'])
id = list(range(len(item_df)))

fb = pd.read_csv('./ml-1m/ml2fb.txt', sep='\t', names=['movieId', 'freebase_id'])

freebase = []
error = 0

for i in range(len(item)):
    try:
        item_id = item_df.loc[item_df['movieId']==item[i], 'movieId'].values[0]
        freebase_name = fb.loc[fb['movieId']==int(item_id), 'freebase_id'].values[0]
        freebase.append(freebase_name)
    except:
        error += 1
        freebase.append('non')

item_list = {
    "org_id" : item,
    "remap_id" : id,
    "freebase_id" : freebase
}
item_list = pd.DataFrame(item_list)

item_list.to_csv('./item_list.txt', sep=' ', index=False)

item_list

,org_id,remap_id,freebase_id
0,1,0,m.0dyb1
1,2,1,m.09w353
2,3,2,m.0676dr
3,4,3,m.03vny7
4,5,4,m.094g2z
...,...,...,...
3701,3948,3701,m.033fqh
3702,3949,3702,m.07k2mq
3703,3950,3703,m.07gv6d
3704,3951,3704,m.0kv6m4


In [6]:
from tqdm import tqdm
# 交互数据remap
t_data = []

for i in tqdm(range(len(user_list))):
    t_data.append(i)    # user: remap_id = index
    org_u = user_list['org_id'][i]

    ratings = df.loc[df['userId']==org_u, 'movieId']
    reamp_list = []
    for j in range(len(ratings)):
        item_reamp = item_list.loc[item_list['org_id']==ratings.values[j], 'remap_id'].values[0]
        reamp_list.append(item_reamp)
    t_data.append(reamp_list)



100%|██████████| 6040/6040 [03:52<00:00, 26.00it/s]


In [7]:
filename = 'tt_data.txt'
file = open('./ml-1m/' + filename,'w+')
for i in range(len(t_data)):
    s = str(t_data[i]).replace('[','').replace(']','')#去除[],这两行按数据不同，可以选择
    s = s.replace("'",'').replace(',','') +'\n'  #去除单引号，逗号，每行末尾追加换行符
    file.write(s)
file.close()

In [8]:
from tqdm import tqdm

lines = open("./ml-1m/tt_data.txt", "r").readlines()
train = []
test = []

for line in tqdm(lines):
    tmps = line.strip()
    inters = [int(i) for i in tmps.split(" ")]

    if len(inters) == 1:
        user_train = [inters[0]]
        user_test = [inters[0]]

    if len(inters) > 1:
        train_tmp = inters[: int(len(inters)*0.8)]
        test_tmp = inters[int(len(inters)*0.8) :]

        user_train.extend(train_tmp)
        user_test.extend(test_tmp)

        train.append(user_train)
        test.append(user_test)


100%|██████████| 12080/12080 [00:00<00:00, 53483.08it/s]


In [9]:
filename = 'train.txt'
file = open('./' + filename,'w+')
for i in range(len(train)):
    s = str(train[i]).replace('[','').replace(']','')#去除[],这两行按数据不同，可以选择
    s = s.replace("'",'').replace(',','') +'\n'  #去除单引号，逗号，每行末尾追加换行符
    file.write(s)
file.close()


In [10]:
filename = 'test.txt'
file = open('./' + filename,'w+')
for i in range(len(test)):
    s = str(test[i]).replace('[','').replace(']','')#去除[],这两行按数据不同，可以选择
    s = s.replace("'",'').replace(',','') +'\n'  #去除单引号，逗号，每行末尾追加换行符
    file.write(s)
file.close()

In [11]:
df

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
